#Understanding Ayushman Bharat



##PROBLEM STATEMENT: Help General public understand the mission of Ayushman Bharat.

##BUSINESS STATEMENT: Develop an AI-powered assistant chatbot to help Government to enable instant resolution to public queries.



###Installing Necessary libraries

In [ ]:
!pip install PyPDF2 -q
!pip install gradio transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00


In [ ]:
import PyPDF2
import requests
from io import BytesIO
import warnings
from urllib3.exceptions import InsecureRequestWarning
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from collections import deque
from torch import autocast

###1. Data Extraction
Excration of the data from the Ayushman Bharat website

In [ ]:
warnings.simplefilter("ignore", InsecureRequestWarning)


def extract_pdf_pages(pdf_url, pages_to_extract):
    response = requests.get(pdf_url, verify=False)
    pdf_file = BytesIO(response.content)


    pdf_reader = PyPDF2.PdfReader(pdf_file)

    extracted_text = ''
    for page_num in pages_to_extract:
        extracted_text += pdf_reader.pages[page_num].extract_text()

    return extracted_text


pdf_url = "https://www.nhm.gov.in/New_Updates_2018/NHM_Components/Health_System_Stregthening/Comprehensive_primary_health_care/letter/Operational_Guidelines_For_CPHC.pdf"


pages_to_extract = [18, 19, 20]


extracted_text = extract_pdf_pages(pdf_url, pages_to_extract)
print(extracted_text[:1999])

AYUSHMAN BHARAT:  Comprehensive Primary Health Care through Health and Wellness CentersThe National Health Mission (NHM), the country’s flagship health systems strengthening programme, 
particularly for primary and secondary health care envisages “attainment of universal access to equitable, 
affordable and quality health care which is accountable and responsive to the needs of people” . Investments 
during the life of the NHM in its earlier phases were targeted to strengthen Reproductive and Child Health (RCH) services and contain the increasing burden of communicable diseases such as Tuberculosis, HIV/AIDS and vector borne diseases. While such a focus on selective primary health care interventions, enabled 
improvements in key indicators related to RCH and select communicable diseases, the range of services 
delivered at the primary care level did not consider increasing disease burden and rising costs of care on account of chronic diseases. 
Studies show that 11.5% households in rur

###Chatbot Development and Gradio Implementation

In [ ]:
model_name = "distilgpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer)
conversation_history = deque(maxlen=3)

cache = {}

def respond_to_query(user_input):
    if user_input in cache:
        return cache[user_input]

    with autocast("cuda"):
        response = chatbot(user_input, max_length=200)[0]['generated_text']

    cache[user_input] = response

    conversation_history.append(user_input)

    return response

def is_ayushman_bharat_related(question):
    relevant_keywords = ['Ayushman Bharat', 'healthcare', 'mission', 'CPHC']
    question_lower = question.lower()
    return any(keyword.lower() in question_lower for keyword in relevant_keywords)

def respond_with_guardrails(user_input):
    if not is_ayushman_bharat_related(user_input):
        return "Sorry, I can only answer questions related to Ayushman Bharat."

    respons = respond_to_query(user_input)
    return respons


iface = gr.Interface(fn=respond_to_query, inputs="text", outputs="text")

iface.launch()


Device set to use cuda:0


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://80e9c903978fa2a1ec.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
